In [13]:
!pip install pyjwt


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import requests
import json
import time
import jwt
import sys
import re
sys.path.append('..')

In [2]:
GOOGLE_KEY_PATH = "google_api.json"
GOOGLE_OAUTH2_TOKEN_URL = "https://oauth2.googleapis.com/token"

In [3]:
from llm_class import LLM

llm = LLM(model_id="bielik", key_path='../key.txt')

In [4]:
with open("../google_api.json") as f:
    google_creds = json.load(f)

In [5]:
def get_access_token():
    issued_at = int(time.time())
    expiration_time = issued_at + 3600

    payload = {
        "iss": google_creds["client_email"],  # Issuer (Service Account Email)
        "scope": "https://www.googleapis.com/auth/cloud-translation",  # Translation API scope
        "aud": GOOGLE_OAUTH2_TOKEN_URL,
        "iat": issued_at,  # Issued At
        "exp": expiration_time  # Expiry
    }

    # Create a JWT signed with the private key
    private_key = google_creds["private_key"]
    jwt_token = jwt.encode(payload, private_key, algorithm="RS256")

    # Exchange JWT for OAuth 2.0 access token
    token_request_data = {
        "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
        "assertion": jwt_token
    }

    response = requests.post(GOOGLE_OAUTH2_TOKEN_URL, data=token_request_data)
    
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Error obtaining access token: {response.text}")

def google_translate(text):
    token = get_access_token()
    url = "https://translation.googleapis.com/language/translate/v2"

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    payload = {
        "q": text,
        "target": "pl"
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()["data"]["translations"][0]["translatedText"]
    else:
        return f"Error: {response.text}"

In [6]:
df = pd.read_csv("../data/dialog_witcher_dataset.csv", delimiter="|")
with open('../data/speakers_characteristics.json', 'r') as f:
    speaker_data = json.load(f) 

In [7]:
def extract_translation(llm_resposne):
    if llm_resposne is None:
        return None
    match = re.search(r'\[(.*?)\]', llm_resposne)
    if match:
        return match.group(1)
    return None

In [8]:
prompt_default = """
Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: {en_text}
"""

In [9]:
prompt_multiple_lines = """ 
Twoim zadaniem jest przetłumaczenie wypowiedzi z języka angielskiego na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
Zdanie po angielsku: I am a very good artist.
Tłumaczenie: [Jestem bardzo dobrym artystą.]

Zdanie po angielsku: Have you heard from her recently?
Tłumaczenie: [Miałeś z nią ostatnio kontakt?]

Zdanie po angielsku: I heard that Michael is going to be a father.
Tłumaczenie: [Słyszałem, że Michael będzie ojcem.]


Oto kontekst: Poniżej znajduje się kawałek dialogu{speakers} 

{dialog}

Na podstawie podanego dialogu, kontekstu oraz podanych wyżej przykładów, przetłumacz tylko i wyłącznie wypowiedź numer {numer} z języka angielskiego na język polski, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych.
Napisz tylko i wyłącznie końcową wersje tłumaczenia w kwadratowych nawiasach z dialogu powyżej. Nie wypisuj nic więcej.
Jeżeli wypowiedź jest w innym języku niż angielski, tylko ją przepisz w nawiasach kwadratowych:

"""

In [10]:
few_shot_single_line = """ 
Wypowiedź mówiona jest przez Geralt (mężczyzna)

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: I am a very good artist.
Przykładowe tłumaczenie: [Jestem bardzo dobrym artystą.]


Wypowiedź mówiona jest przez Yennefer (kobieta)

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: Dubhenn haern am glâdeal.
Przykładowe tłumaczenie: [Dubhenn haern am glâdeal.]


Wypowiedź mówiona jest przez Random person

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: Have you heard from her recently?
Przykładowe tłumaczenie: [Miałeś z nią ostatnio kontakt?]


Wypowiedź mówiona jest przez Troll

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: Soup is very yummy, I want more.
Przykładowe tłumaczenie: [Zupa być pyszna, chcieć więcej]
"""

In [11]:
prompt_single_line = """
Twoim zadaniem jest przetłumaczenie wypowiedzi na język polski, dostosowując tłumaczenie pod zadany kontekst, a następnie napisanie końcowej wersji tłumaczenia w nawiasach kwadratowych.

Oto kilka przykładów jak powinno wyglądać końcowe tłumaczenie w kwadratowych nawiasach:
{examples}


Oto kontekst: Wypowiedź mówiona jest przez {speaker}{gender}.

Przetłumacz podaną wypowiedź na język polski uwzgledniając podany kontekst, a końcową wersję tłumaczenia zapisz w nawiasach kwadratowych: {text_to_translate}
"""

In [12]:
def choose_and_format_prompt(speaker1, speaker2, speaker3, prev_text, text_to_translate, next_text):

    if speaker2 == None:
        print("no main speaker")
        return None
    
    speakers = [speaker1, speaker2, speaker3]
    non_none_speakers = [s for s in speakers if s is not None]
    unique_speakers = set(non_none_speakers)
    is_monolog = len(unique_speakers) == 1
    
    speaker2_gender = speaker_data.get(speaker2, {}).get('gender')
    speaker2_gender = f" ({speaker2_gender})" if speaker2_gender and speaker2_gender.lower() != 'unknown' else ""


    #jezeli pojedyncza wypowiedz
    if is_monolog:
        return prompt_single_line.format(speaker=speaker2, gender=speaker2_gender, text_to_translate=text_to_translate, examples=few_shot_single_line)
    
    #jezeli weicej niz jedna wypowiedz
    speaker1_gender = speaker_data.get(speaker1, {}).get('gender')
    speaker1_gender = f" ({speaker1_gender})" if speaker1_gender and speaker1_gender.lower() != 'unknown' else ""

    speaker3_gender = speaker_data.get(speaker3, {}).get('gender')
    speaker3_gender = f" ({speaker3_gender})" if speaker3_gender and speaker3_gender.lower() != 'unknown' else ""

    speakers = ""
    dialog = ""


    def number_to_st(i):
        if i == 1:
            return "1."
        elif i == 2:
            return "2."
        else:
            return "3."
    i=1
    numer = 1

    if speaker1 and speaker1 != speaker2:
        dialog += f"{number_to_st(i)} {speaker1} - {prev_text}\n"
        numer = i
        i += 1

    if speaker2:
        dialog += f"{number_to_st(i)} {speaker2} - {text_to_translate}\n"
        numer = i
        i += 1
    if speaker3 and speaker3 != speaker2:
        dialog += f"{number_to_st(i)} {speaker3} - {next_text}\n"

  
    if (speaker1 != speaker2 and speaker3 == None) or (speaker1 != speaker2 and speaker3 == speaker2) or (speaker1 != speaker2 and speaker1==speaker3):
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender}"
    elif (speaker3 != speaker2 and speaker1 == None) or (speaker3 != speaker2 and speaker1 == speaker2) or (speaker3 != speaker2 and speaker3==speaker1):
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker3}{speaker3_gender}"
    else:
        speakers = f", rozmawia w nim {speaker2}{speaker2_gender} z {speaker1}{speaker1_gender} oraz z {speaker3}{speaker3_gender}"

    return prompt_multiple_lines.format(speakers=speakers, dialog=dialog, text_to_translate=text_to_translate, numer=numer)

In [13]:
df

,speaker,prev_text,prev_speaker,next_text,next_speaker,en_text,pl_text
0,Geralt,Greetings. Come to look over my spring collect...,Elihal,"Welcome to the Seven Cats, the luckiest tavern...",Novigrad Poor Woman 01,So long.,Bywaj.
1,Temerian Bandit 02,No!,Temerian Bandit 02,Ehhh!,Temerian Bandit 02,Come to pappy!,Chodź do wujka!
2,Temerian Bandit 02,Come to pappy!,Temerian Bandit 02,I don't wanna die.,Temerian Bandit 02,Ehhh!,Eh!
3,Temerian Bandit 02,Still?,Temerian Bandit 02,Rip his stones off!,Temerian Bandit 02,Your arse is mine.,Twoja dupa należy teraz do mnie.
4,Temerian Bandit 03,I don't wanna die.,Temerian Bandit 02,Fuck.,Temerian Bandit 03,Hoo-ahh!,Hu-ha!
...,...,...,...,...,...,...,...
33728,Avallach,Ciri!,Avallach,I'll not stop with his staff.,Cirilla,The ice… it's a spell. You must find the mage ...,Ten lód... To zaklęcie. Musisz zniszczyć kostu...
33729,Ida Emean,Stop twisting my words.,Yennefer,"Ceádmil, Aen Saevherne.",Geralt,"Cáed'mil, Yennefer aep Vengerberg. Gwynbleidd.","Ceádmil, Yennefer aep Vengerberg. Gwynbleidd."
33730,Geralt,NaN,NaN,Tis but the start. Real challenge awaits at th...,Lugos The Glaucous,"If there's one thing I'm not afraid of, it's n...","Czego jak czego, ale nekkerów to się akurat ni..."
33731,Geralt,Shame to let such a lad waste away behind bars...,Skellige Guard 04,NaN,NaN,And they say violence is never the answer…,"A mówią, że przemoc nigdy nie jest rozwiązanie..."


In [14]:
sampled_df = df.sample(n=10000, random_state=42).reset_index(drop=True)

for index, row in sampled_df.iterrows():
    current_speaker = row['speaker']
    text_to_translate = row['en_text']
    prev_speaker = row['prev_speaker']
    prev_text = row['prev_text']
    next_speaker = row['next_speaker']
    next_text = row['next_text']


    #dialog
    translation = None
    i = 0

    while translation == None and i < 3:
        prompt = choose_and_format_prompt(speaker1=prev_speaker, speaker2=current_speaker, speaker3=next_speaker, prev_text=prev_text, text_to_translate=text_to_translate, next_text=next_text)
        response = llm.prompt_chat_custom_temperature(prompt=prompt, temperature=0.0)
        translation = extract_translation(response)
        i+=1

    if translation == None:
        print(prompt)
        sampled_df.at[index, f'dialog'] = "ERROR"
        continue
        
    sampled_df.at[index, f'dialog'] = translation

    #zwykly prompt
    response_default = llm.prompt_chat_custom_temperature(prompt=prompt_default.format(en_text=text_to_translate), temperature=0.0)
    sampled_df.at[index, f'default'] = response_default

    #google translate
    google_translation = google_translate(text_to_translate)
    sampled_df.at[index, f'google'] = google_translation


    if (index + 1) % 100 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df)} samples...")

sampled_df.to_csv("results/final_test.csv", sep="|", index=False)

Processed 100 out of 10000 samples...
Processed 200 out of 10000 samples...
Processed 300 out of 10000 samples...
Processed 400 out of 10000 samples...
Processed 500 out of 10000 samples...
Processed 600 out of 10000 samples...
Processed 700 out of 10000 samples...
Processed 800 out of 10000 samples...
Processed 900 out of 10000 samples...
Processed 1000 out of 10000 samples...
Processed 1100 out of 10000 samples...
Processed 1200 out of 10000 samples...
Processed 1300 out of 10000 samples...
Processed 1400 out of 10000 samples...
Processed 1500 out of 10000 samples...
Processed 1600 out of 10000 samples...
Processed 1700 out of 10000 samples...
Processed 1800 out of 10000 samples...
Processed 1900 out of 10000 samples...
Processed 2000 out of 10000 samples...
Processed 2100 out of 10000 samples...
Processed 2200 out of 10000 samples...
Processed 2300 out of 10000 samples...
Processed 2400 out of 10000 samples...
Processed 2500 out of 10000 samples...
Processed 2600 out of 10000 sample